In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config Completer.use_jedi = False  # to make autocompletion faster

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import gc
%matplotlib


In [ ]:
tf.__version__

In [ ]:
"""
!kaggle competitions download -c web-traffic-time-series-forecasting
!yes|unzip web-traffic-time-series-forecasting.zip
!mkdir logs
"""

## Make dataset

In [ ]:
from webtraffic_utils import *
output_len = 62

In [ ]:
# features like (access, spectral tones) created by feature_engineering.ipynb

from ast import literal_eval
df_feat = pd.read_csv("features_computed.csv.zip",converters={"tones":literal_eval}).set_index("Page")
weekly_tone = df_feat["tones"].apply(lambda x: (np.abs(np.array(x)-1./7.)<1e-2).any()).rename("week")

In [ ]:
df_ds = pd.read_csv("train_2.csv.zip", header=0).set_index("Page").fillna(0).astype(np.int32)

[repeat last value](#repeat-last-value)  
[linear model](#linear-model)  
[RNN](#RNN)  

## Exploration

In [ ]:
df_feat["access"].value_counts()

In [ ]:
# remarkable pages

rem_pages = [
    'Acier_inoxydable_fr.wikipedia.org_desktop_all-agents',
]
#rem_pages = df_ds.loc[weekly_tone & (df_feat["access"] == "desktop_all-agents")].index



page = rem_pages[0]
traffic_t =  df_ds.loc[page].values[:-1].astype(int)
f,vax = plt.subplots(1,3, figsize=(20,4))
fax = vax.flat


ax=next(fax)
ax.plot(traffic_t)
ax.set_title("time traffic")


ax=next(fax)
ax.plot(estimated_autocorrelation(traffic_t))
ax.set_xticks([0,365,2*365])
ax.set_title("autocorrelation")
ax.grid()

ax=next(fax)
plot_spectrest(traffic_t, ax)
ax.set_title("spectral estimation")




plt.suptitle(page, fontsize=15)
plt.show()

## Models

[repeat last value](#repeat-last-value)  
[linear model](#linear-model)  
[RNN](#RNN)  

### repeat last value

In [ ]:
import tensorflow_probability as tfp

class Median(tf.keras.layers.Layer):
    def __init__(self, median_depth=40):
        super().__init__()
        self.median_depth =median_depth
            
    def call(self, inputs):
        #print(inputs)
        Xtraff = tfp.stats.percentile(inputs[:,-self.median_depth:], 50.0, 
                                      interpolation='lower', axis=1)
        return tf.tile(tf.expand_dims(Xtraff,axis=1), [1,output_len])

In [ ]:
I_traffic = tf.keras.layers.Input(shape=(None,))
I_page = tf.keras.layers.Input(shape=(), dtype=object)
outputs = Median(40)(I_traffic)  
med = tf.keras.models.Model([I_page,I_traffic],outputs)

med.compile(loss=SmapeLoss(), metrics=[SmapeMetric()])

features, target = get_model_inputs(df_ds)
med.evaluate(features, target, batch_size=1000)

med.save("saved_model/median", save_format='tf')
_=gc.collect()

In [ ]:
features, target = get_model_inputs(df_ds.loc[~weekly_tone])
med.evaluate(features, target, batch_size=1000)


In [ ]:
plot_check_result(df_ds, rem_pages[0], [med])

### linear model

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.05)
lr_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
tf.keras.backend.clear_session()
tb_cb = create_tb_cb("linear")
Ldelay=100

traffic = tf.keras.layers.Input(shape=(1000,))
page = tf.keras.layers.Input(shape=())

outputs = tf.keras.layers.Dense(units=output_len,input_dim=Ldelay)(traffic[:,-Ldelay:])

model_linear = tf.keras.Model(inputs=[page, traffic], outputs=[outputs])

model_linear.summary()

model_linear.compile(loss=SmapeLoss(), optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
                     metrics=[SmapeMetric()])

In [ ]:
es_cb = tf.keras.callbacks.EarlyStopping(monitor='smape', min_delta=0.1, patience=5, verbose=0, restore_best_weights=True)

#ds1.cache()
features, target = get_model_inputs(df_ds[weekly_tone])
feat_val, target_val = get_model_inputs(df_ds.iloc[:,:-50][weekly_tone])
model_linear.fit(features, target, epochs=100, callbacks=[tb_cb, es_cb], batch_size=32, validation_data=(feat_val, target_val))
model_linear.save("saved_model/model_linear_nw", save_format='tf')

In [ ]:
plot_check_result(df_ds, rem_pages[0], [model_linear, med])

In [ ]:
weights = model_linear.get_layer("dense").get_weights()[0]

f,ax = plt.subplots()
ax.plot(np.abs(weights[:,0]))
ax.grid()

### mixed model

In [ ]:
def smape_row(df_train, model):
    features, ytrue = get_model_inputs(df_train)
    def _smape_row(A, F):
        return np.mean(100 * (2 * np.abs(F - A) / (np.abs(A) + np.abs(F) + np.finfo(float).eps)), axis=1)
    return pd.Series(_smape_row(model.predict(features, batch_size=1000, verbose=1), ytrue), index=df_train.index)

In [ ]:
# find best choice between linear and median
cv_sz = 1
decal = 162

best_accu = pd.Series(0, index=df_ds.index)
for ii in range(cv_sz):
    last_samp = df_ds.shape[1]-1-ii*decal
    df_train = df_ds.iloc[:,last_samp-decal:last_samp]
    smape_lin = smape_row(df_train, model_linear).rename("smape_lin")
    smape_med = smape_row(df_train, med).rename("smape_med")
    print("fold : {}, mixed smape : {:.1f}".format( ii, pd.concat([smape_lin, smape_med], axis=1).min(axis=1).mean()))
    best_accu = best_accu + (pd.concat([smape_lin, smape_med], axis=1).idxmin(axis=1) == "smape_med").astype(float)/cv_sz
    
_ = gc.collect()


In [ ]:
median_en = best_accu > 0.5
smape_lin = smape_row(df_ds.iloc[:,:], model_linear).rename("smape_lin")
smape_med = smape_row(df_ds.iloc[:,:], med).rename("smape_med")
smape_mixed = smape_med* median_en + smape_lin * (1-median_en)
smape_mixed.mean()

In [ ]:
tf.keras.backend.clear_session()
tb_cb = create_tb_cb("mixed")

traffic = tf.keras.layers.Input(shape=(1000,))
page = tf.keras.layers.Input(shape=())
algo_select = tf.keras.layers.Input(shape=())
o_lin = model_linear.get_layer("dense")(traffic[:,-Ldelay:])
o_med = Median(40)(traffic)
    
outputs =  o_lin * tf.tile(tf.expand_dims(tf.cast(1-algo_select, o_lin.dtype),1),[1,62])
outputs =  outputs + o_med * tf.tile(tf.expand_dims(tf.cast(algo_select, o_lin.dtype),1),[1,62])

    
mixed_model = tf.keras.Model(inputs=[page, traffic, algo_select], outputs=[outputs])

mixed_model.compile(loss=SmapeLoss(), optimizer=tf.optimizers.Adam(learning_rate=1e-4), 
                     metrics=[SmapeMetric()])


In [ ]:
features, target = get_model_inputs(df_ds.join(median_en.rename("feat_select")))
mixed_model.evaluate(features,target)

In [ ]:
mixed_model.save("saved_model/mixed_model", save_format='tf')

In [ ]:
gc.collect()

In [ ]:
plot_check_result(df_ds.join(median_en.rename("feat_select")), rem_pages[0], [mixed_model, med])

### RNN

In [ ]:
#!rm -Rf logs/*

In [ ]:
Nneurons = 20
Nlayers = 1
MaxTs = 150
usePastYear = False
useMetadata = False
Seq2seq = True


In [ ]:
class normalize_rnn(tf.keras.layers.Layer):
    def call(self, inputs):
        fact = tf.reduce_max(inputs, axis=1, keepdims=True)
        ret = tf.divide(inputs, fact + 1e-10) 
        return ret, fact

In [ ]:
voc_access = np.unique(df_ds.index.map(lambda x: "_".join(x.split("_")[-2:])))
voc_project = np.unique(df_ds.index.map(lambda x: x.split("_")[-3]))
onehotAccess = OneHotEncodingLayer(voc_access, name="ohAccess")
onehotProject = OneHotEncodingLayer(voc_project, name="ohProject")

In [ ]:
tf.random.set_seed(42)

simn = 'Ts'+str(MaxTs)+'-Nn'+str(Nneurons)+'-Nl'+str(Nlayers)

tf.keras.backend.clear_session()
tb_cb = create_tb_cb(simn)

class preprocessing_rnn(tf.keras.layers.Layer):
    def call(self, inputs, access1h):
        ret = inputs[:,-MaxTs:,np.newaxis]
        if useMetadata:
            access_broadcast = tf.tile(access1h[:,np.newaxis,:],[1,MaxTs,1])
            ret = tf.concat([ret, access_broadcast], axis=2)
            
        if usePastYear:
            pastYear = inputs[:, -MaxTs-365+output_len:-365+output_len, np.newaxis]
            ret = tf.concat([ret, pastYear], axis=2)
        return ret

I_page = tf.keras.layers.Input(shape=(), dtype=object)
I_traffic = tf.keras.layers.Input(shape=(None,))
weekly = tf.keras.layers.Input(shape=())

access1h = onehotAccess(I_page)

x, factors = normalize_rnn()(I_traffic)
x = preprocessing_rnn()(x, access1h)
for ii in range(Nlayers-1):
    x = tf.keras.layers.GRU(Nneurons, return_sequences=True)(x)
x = tf.keras.layers.GRU(Nneurons, return_sequences=Seq2seq>0)(x)

if not Seq2seq:
    x= tf.keras.layers.Dense(output_len)(x)
else:
    x = keras.layers.TimeDistributed(keras.layers.Dense(output_len))(x)
    factors = tf.cast(tf.tile(tf.expand_dims(factors, axis=1), (1, MaxTs, output_len)), tf.float32)

outputs= tf.multiply(x, factors)

model_rnn = tf.keras.Model(inputs=[I_page, I_traffic, weekly], outputs=[outputs])

model_rnn.summary()

In [ ]:
model_rnn.compile(loss=SmapeLoss(), optimizer=tf.optimizers.Adam(learning_rate=1e-3), metrics=[SmapeMetric()])

In [ ]:
es_cb = tf.keras.callbacks.EarlyStopping(monitor='smape', min_delta=0.1, patience=5, verbose=0, restore_best_weights=True)

weekly_pages = df_ds.loc[weekly_tone & (df_feat["access"] == "desktop_all-agents")].index

features, target = get_model_inputs(df_ds.iloc[:,:-62].loc[weekly_tone], return_seq=Seq2seq)
feat_val, target_val = get_model_inputs(df_ds.loc[weekly_tone], return_seq=Seq2seq)

model_rnn.fit(features, target, epochs=100, callbacks=[tb_cb, es_cb] , batch_size=1, validation_data=(feat_val, target_val))

del feat_val, target_val, features, target
gc.collect()

 3711/17323 [=====>........................] - ETA: 5:06 - loss: 30.6071 - smape: 30.6936

In [ ]:
from tqdm import tqdm

def chunk_smape(df_ds, chunk=10000):
    smape_v=[]
    for ii in tqdm(range(len(df_ds)//chunk+1)):
        features, target = get_model_inputs(df_ds.iloc[ii*chunk:(ii+1)*chunk,:], return_seq=False)
        smape_v.append(SmapeLoss()(model_rnn.predict(features, batch_size=1000)[:,-1,:], target).numpy())
    return np.mean(smape_v)

In [ ]:
chunk_smape(df_ds.loc[~weekly_tone])

In [ ]:
model_rnn.save("saved_model/model_rnn")

In [ ]:
plot_check_result(df_ds.iloc[:,:], rem_pages[0], [model_rnn])

### ouput

In [ ]:
key = pd.read_csv("key_2.csv.zip").set_index("Page")

In [ ]:
key["Visits"] = 0

In [ ]:
def output_form(features, _model=None):
    """ return a serie indexed by Page """
    out_date = pd.date_range(start="2017-09-13", end="2017-11-13", freq="1D").strftime("%Y-%m-%d").to_list()
    num_pred = np.clip(_model.predict(features, batch_size=1000, verbose=1).astype(int), a_min=0, a_max=None)
    index = df_ds.index  # np.char.decode(features[0].numpy().astype(bytes))
    ret = pd.DataFrame(num_pred, columns=out_date, index=index).stack().rename("Visits")
    ret.index = [ii[0]+"_"+ii[1] for ii in ret.index]
    return ret

In [ ]:
def output_form(features, _model=None):
    """ return a serie indexed by Page """
    out_date = pd.date_range(start="2017-09-13", end="2017-11-13", freq="1D").strftime("%Y-%m-%d").to_list()
    num_pred = np.clip(_model.predict(features, batch_size=1, verbose=1).astype(int)[:,-1,:] , a_min=0, a_max=None)
    index = df_ds.index  # np.char.decode(features[0].numpy().astype(bytes))
    ret = pd.DataFrame(num_pred, columns=out_date, index=index).stack().rename("Visits")
    ret.index = [ii[0]+"_"+ii[1] for ii in ret.index]
    return ret

features = [tf.convert_to_tensor(df_ds.index), df_ds.values]
Visits_pred = output_form(features, model_rnn)


In [ ]:
features = [tf.convert_to_tensor(df_ds.index), df_ds.values, tf.convert_to_tensor(median_en)]

Visits_pred = output_form(features, mixed_model)
Visits_pred.head()

In [ ]:
key.loc[Visits_pred.index, "Visits"] = Visits_pred.astype(int)

In [ ]:
key.to_csv("subm_mixed.csv", encoding='utf-8', index=False)

In [ ]:
"""
# load kaggle environment if in google colab
from google.colab import files
files.upload() #upload kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
"""

In [ ]:
!kaggle competitions submit -f subm_mixed.csv -m "mixed med/lin (elt wise best)"  web-traffic-time-series-forecasting

In [ ]:
del Visits_pred, key ,features
gc.collect()